In [0]:
# %sql

# select * from dimruncycle
# order by runcycleid desc
# limit 1;

# -- DROP TABLE IF EXISTS dimstages;

# -- CREATE TABLE dimstages
# -- USING DELTA AS
# --     SELECT
# --         stageid,
# --         name,
# --         CAST(NULL AS STRING) AS description,
# --         CASE
# --             WHEN isactive = true THEN 'Y'
# --         ELSE
# --             'N'
# --         END AS cactive,
# --         'FOIMOD' AS sourceoftruth
# --     FROM foi_mod.programareadivisionstages
# --     order by stageid;

In [0]:
%restart_python
%pip install boto3
import boto3
import os
from botocore.exceptions import NoCredentialsError
import datetime
import sys
sys.path.insert(0, '/Workspace/Shared')
import etl_helpers
from pyspark.sql.functions import col #lit

tablename = "dimstages"
runcycleid = etl_helpers.start_run_cycle(tablename)
os.makedirs("/dbfs/foi/dataload", exist_ok=True)  # make sure directory exists

try:
    query = f"""
        SELECT
            stageid,
            name,
            CAST(NULL AS STRING) AS description,
            CASE
                WHEN isactive = true THEN 'Y'
            ELSE
                'N'
            END AS cactive,
            'FOIMOD' AS sourceoftruth
        FROM foi_mod.programareadivisionstages
        order by stageid;
        """

    # print(query)

    df = spark.sql(query)
    df.show()

    # order of columns here is important!
    df_mapped = df.selectExpr(
            "stageid AS stageid",
            "name AS name",
            "description AS description",
            "cactive AS cactive",
            "sourceoftruth AS sourceoftruth"
        )
    df_mapped.show()

    from delta.tables import DeltaTable
    delta_table = DeltaTable.forName(spark, f"hive_metastore.default.{tablename}")
    delta_table.alias("target").merge(
        df_mapped.alias("source"),
        "target.stageid = source.stageid AND target.sourceoftruth = source.sourceoftruth"
    ).whenMatchedUpdate(set = {
        "name": col("source.name"),
        "description": col("source.description"),
        "cactive": col("source.cactive")
    }).whenNotMatchedInsert(values = {
        "stageid": col("source.stageid"),
        "name": col("source.name"),
        "description": col("source.description"),
        "cactive": col("source.cactive"),
        "sourceoftruth": col("source.sourceoftruth")
    }).execute()

    etl_helpers.end_run_cycle(runcycleid, 't', tablename)
except NoCredentialsError:
    print("Credentials not available")
    etl_helpers.end_run_cycle(runcycleid, 'f', tablename, "Credentials not available")
    dbutils.notebook.exit("Error: Something went wrong.")
except Exception as e:
    if (str(e) == "no changes for today"):
        print("here")
        etl_helpers.end_run_cycle(runcycleid, 't', tablename)
    else:
        print(f"An error occurred: {e}")    
        etl_helpers.end_run_cycle(runcycleid, 'f', tablename, f"An error occurred: {e}")
        dbutils.notebook.exit("Error: Something went wrong.")